<a href="https://colab.research.google.com/github/Madesh09/Predicting-Incident-Priorities-Forecasting-Incident-Volume-in-ITSM/blob/main/NOC_ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1.Environment Setup**

In [ ]:
%pip install --quiet --upgrade langchain langchain-community langchain-chroma
%pip install --upgrade --quiet huggingface_hub
%pip install pypdf
%pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 607.0/607.0 kB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.0/408.0 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.9/296.9 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.4 MB/s eta 0:

**2.Load** **Documents**

In [ ]:
from langchain.document_loaders import PyPDFDirectoryLoader

directory = '/content/sample_data/kb_artical'
loader = PyPDFDirectoryLoader(directory)
documents = loader.load()

documents

[Document(metadata={'source': '/content/sample_data/kb_artical/eNOC Actions for Windows Server.pdf', 'page': 0}, page_content='eNOC Actions for Windows Server: \nThis document provides step-by-step guidelines for the eNOC team on alert handling various Windows Server \nevents, including CPU/Memory Utilization, Windows Server Down, and Disk Space Usage. \n1. CPU or Memory Utilization \n• Validate current levels using monitoring tools. \n• Monitor for significant changes or trends. \n• If high levels persist, escalate to the Windows or Application Team. \n• Determine the application tier: \n• Tier 0 or 1: Immediately call the appropriate team. \n• Other Tiers: No call needed, assign to the appropriate team, and send an email notification. \nDocument findings and actions in the incident management system. \n2. Windows Server Down \nApproved Change or Patch Schedule \n• Downgrade P2 Incident to P3. \n• Notify OM (Outage Management) via email. \n• Resolve the P3 Incident with findings. \n• 

**3. Count Words in Documents**

In [ ]:
# counting the number words in the loaded doc
def count_words_in_documents(documents):
    word_counts = []
    for doc in documents:
        word_count = len(doc.page_content.split())
        word_counts.append(word_count)
    return word_counts

# Example usage
word_counts = count_words_in_documents(documents)
word_counts

[314]

**4. Text Splitting**

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_docs(documents,chunk_size=500,chunk_overlap=10):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  docs = text_splitter.split_documents(documents)
  return docs

docs = split_docs(documents)
print(len(docs))

5


**5. Embedding Generation**

In [ ]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2",encode_kwargs={"normalize_embeddings": True},)

**6. Vector Store Creation with Chroma**

In [ ]:
from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(documents= docs, embedding=embeddings)

**7. Setting up the Retriever**

In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})


In [ ]:
# get a token: https://huggingface.co/docs/api-inference/quicktour#get-your-api-token

from getpass import getpass

HUGGINGFACEHUB_API_TOKEN = getpass()

··········


In [ ]:
from langchain_community.llms import HuggingFaceEndpoint
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate

**8. Prompt Template Definition**

In [ ]:
from langchain.prompts.prompt import PromptTemplate


template_string="""
<system>\
You are a honest NOC assistant agent and your job is giving answers based on content. If user ask any question is not related to content then say "I don't know, Please ask anything about NOC". The answer should be in simple statement.
</system>
###
content: {content}
####
<user>{text}</user>
###
answer:
"""


prompt = PromptTemplate.from_template(template=template_string)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

**9. LLM Initialization**

In [ ]:
llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.2",
    max_length=128,
    temperature=0.5,
    huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN,
)

<ipython-input-19-e4a5d59e79cc>:1: LangChainDeprecationWarning: The class `HuggingFaceEndpoint` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  llm = HuggingFaceEndpoint(
                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.


**10. Query Processing**

In [ ]:
query = "Process for Memory alert ?"

In [ ]:
retrieved_docs = retriever.invoke(query)

len(retrieved_docs)

5

**11. Running the LLM Chain**

In [ ]:
%%time
from langchain.chains import LLMChain

chain = LLMChain(
    llm=llm,
    prompt=prompt,
    output_parser=output_parser
)

res=chain.invoke({"content":retrieved_docs,"text":query})
res['text']

CPU times: user 21.3 ms, sys: 4.68 ms, total: 26 ms
Wall time: 2.65 s


'The process for handling a CPU alert involves validating the current CPU levels using monitoring tools, monitoring for significant changes or trends, and if high levels persist, escalating to the Windows or Application Team. If the application tier is Tier 0 or 1, call the appropriate team immediately. For other tiers, no call is needed, assign to the appropriate team, and send an email notification. Document findings and actions in the incident management system.'